In [1]:
import pandas as pd

In [2]:
dataSet = pd.read_csv('CKD.csv')

In [3]:
dataSet.head()

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.0200,1.0,0.000000,121.000000,36.0,1.2,137.528754,4.627244,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,9.0,55.0,1.0200,4.0,0.000000,148.036517,18.0,0.8,137.528754,4.627244,...,38,6000,5.2,no,no,no,good,no,no,ckd
2,62.0,80.0,1.0100,2.0,1.125356,223.500000,53.0,1.8,137.528754,4.627244,...,31,7500,5.2,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.0075,4.0,0.000000,117.000000,56.0,3.8,126.000000,2.800000,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.0100,2.0,0.000000,106.000000,26.0,1.4,137.528754,4.627244,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [4]:
dataSet['classification'].value_counts()

ckd       248
notckd    150
ckd\t       2
Name: classification, dtype: int64

In [5]:
dataSet.loc[dataSet['classification'] == 'ckd', 'classification'] = 'yes'
dataSet.loc[dataSet['classification'] == 'notckd', 'classification'] = 'no'
dataSet.loc[dataSet['classification'] == 'ckd\t', 'classification'] = 'yes'

In [6]:
dataSet['classification'].value_counts()

yes    250
no     150
Name: classification, dtype: int64

In [7]:
dataSet = pd.get_dummies(dataSet, drop_first = True)

In [8]:
dataSet.head()

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,dm_\tyes,dm_ yes,dm_no,dm_yes,cad_no,cad_yes,appet_poor,pe_yes,ane_yes,classification_yes
0,48.0,80.0,1.0200,1.0,0.000000,121.000000,36.0,1.2,137.528754,4.627244,...,0,0,0,1,1,0,0,0,0,1
1,9.0,55.0,1.0200,4.0,0.000000,148.036517,18.0,0.8,137.528754,4.627244,...,0,0,1,0,1,0,0,0,0,1
2,62.0,80.0,1.0100,2.0,1.125356,223.500000,53.0,1.8,137.528754,4.627244,...,0,0,0,1,1,0,1,0,1,1
3,48.0,70.0,1.0075,4.0,0.000000,117.000000,56.0,3.8,126.000000,2.800000,...,0,0,1,0,1,0,1,1,1,1
4,51.0,80.0,1.0100,2.0,0.000000,106.000000,26.0,1.4,137.528754,4.627244,...,0,0,1,0,1,0,0,0,0,1


In [9]:
independent = dataSet.iloc[:,:len(dataSet.columns)-1]
dependent = dataSet.iloc[:,len(dataSet.columns)-1:]

In [10]:
independent.head(2)

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,htn_yes,dm_\tyes,dm_ yes,dm_no,dm_yes,cad_no,cad_yes,appet_poor,pe_yes,ane_yes
0,48.0,80.0,1.02,1.0,0.0,121.000000,36.0,1.2,137.528754,4.627244,...,1,0,0,0,1,1,0,0,0,0
1,9.0,55.0,1.02,4.0,0.0,148.036517,18.0,0.8,137.528754,4.627244,...,0,0,0,1,0,1,0,0,0,0


In [11]:
dependent

,classification_yes
0,1
1,1
2,1
3,1
4,1
...,...
395,0
396,0
397,0
398,0


# feature selection

In [12]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
selectFeature = SelectKBest(score_func = chi2, k = 4)
selectFeature = selectFeature.fit(independent, dependent)
selectColumnsIndex = selectFeature.get_support(indices=True)
selectColumnsIndex # select features

array([3, 5, 6, 7], dtype=int64)

In [13]:
independent = independent[independent.columns[[selectColumnsIndex]]]

C:\Users\REVANTH KUMAR\Anaconda3\envs\hopeAi\lib\site-packages\pandas\core\indexes\base.py:4291: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [14]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(independent, dependent, test_size = 0.25, random_state=0)

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xTrain = sc.fit_transform(xTrain)
xTest = sc.transform(xTest)

In [16]:
from sklearn.ensemble import RandomForestClassifier # Random forest classifier

In [17]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(xTrain, yTrain)

C:\Users\REVANTH KUMAR\Anaconda3\envs\hopeAi\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [18]:
def cm_prediction(classifier, xTest, yTest):
    yPred = classifier.predict(xTest)
    
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(yTest, yPred)    
    
    from sklearn.metrics import accuracy_score 
    from sklearn.metrics import classification_report 
    
    Accuracy=accuracy_score(yTest, yPred )
    
    report = classification_report(yTest, yPred)
    return classifier, Accuracy, report, xTest, yTest, cm

In [19]:
classifier, Accuracy, report, xTest, yTest, cm = cm_prediction(classifier, xTest, yTest)

In [20]:
classifier

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [21]:
Accuracy

0.96

In [22]:
print(report)

              precision    recall  f1-score   support

           0       0.97      0.92      0.95        38
           1       0.95      0.98      0.97        62

    accuracy                           0.96       100
   macro avg       0.96      0.95      0.96       100
weighted avg       0.96      0.96      0.96       100



In [23]:
cm

array([[35,  3],
       [ 1, 61]], dtype=int64)

# save the model

In [24]:
import pickle 

In [25]:
fileName = 'ckdPrediction.sav'

In [26]:
pickle.dump(classifier, open(fileName, 'wb'))

# save the Standard Scaler for transform the values

In [27]:
fileName = 'ckdStandardScaler.sav'

In [28]:
pickle.dump(sc, open(fileName, 'wb'))